In [ ]:
!pip install "git+https://github.com/wavefrontshaping/complexPyTorch.git"

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from math import cos as cos
from math import sin as sin
from torch.utils.data import TensorDataset
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from complexPyTorch.complexLayers import ComplexConv2d, ComplexLinear, ComplexMaxPool2d
from complexPyTorch.complexFunctions import complex_max_pool2d, complex_dropout

In [ ]:
grid_size = 7
image_size = (64, 64)
num_BB = 2
num_classes = 20
depth = 5 * num_BB + num_classes
num_epoches = 2
Batch_size = 8

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
class LeakyReLUComplex(nn.Module):
    def __init__(self, negative_slope=0.01):
        super(LeakyReLUComplex, self).__init__()
        self.negative_slope = negative_slope

    def forward(self, input):
        input.real = torch.where(input.real > 0, input.real, input.real * self.negative_slope)
        input.imag = torch.where(input.imag > 0, input.imag, input.imag * self.negative_slope)
        return input

In [ ]:
class ClampComplex(nn.Module):
  def __init__(self, min, max):
    super(ClampComplex, self).__init__()
    self.min = min
    self.max = max

  def forward(self, input):
    input.real = torch.clamp(input.real, min=self.min, max=self.max)
    input.imag = torch.clamp(input.imag, min=self.min, max=self.max)
    return input

In [ ]:
class ComplexDropout(nn.Module):
    def __init__(self, p=0.5):
        super().__init__()
        self.p = p

    def forward(self, input):
        if self.training:
            return complex_dropout(input, self.p)
        else:
            return input

In [ ]:
class YOLOv1(nn.Module):
    def __init__(self):
      super().__init__()
      # self.clampLayer = ClampComplex(min=0, max=1)
      layer1 = [
          ComplexConv2d(3, 32, kernel_size=7, stride=2, padding=3),
          LeakyReLUComplex(negative_slope=0.1)]

      layer2 = [ComplexConv2d(32, 96, kernel_size=3, padding=1),
          LeakyReLUComplex(negative_slope=0.1)]

      layer3 = [ComplexConv2d(96, 64, kernel_size=1),
          LeakyReLUComplex(negative_slope=0.1),
          ComplexConv2d(64, 128, kernel_size=3, padding=1),
          LeakyReLUComplex(negative_slope=0.1),
          ComplexConv2d(128, 128, kernel_size=1),
          LeakyReLUComplex(negative_slope=0.1),
          ComplexConv2d(128, 256, kernel_size=3, padding=1),
          LeakyReLUComplex(negative_slope=0.1)]

      layer4 = [ComplexConv2d(256, 128, kernel_size=1),
          ComplexConv2d(128, 256, kernel_size=3, padding=1),
          LeakyReLUComplex(negative_slope=0.1),
          ComplexConv2d(256, 128, kernel_size=1),
          ComplexConv2d(128, 256, kernel_size=3, padding=1),
          LeakyReLUComplex(negative_slope=0.1),
          ComplexConv2d(256, 128, kernel_size=1),
          ComplexConv2d(128, 256, kernel_size=3, padding=1),
          LeakyReLUComplex(negative_slope=0.1),
          ComplexConv2d(256, 128, kernel_size=1),
          ComplexConv2d(128, 256, kernel_size=3, padding=1),
          LeakyReLUComplex(negative_slope=0.1),
          ComplexConv2d(256, 256, kernel_size=1),
          ComplexConv2d(256, 512, kernel_size=3, padding=1),
          LeakyReLUComplex(negative_slope=0.1)]

      layer5 = [ComplexConv2d(512, 256, kernel_size=1),
          ComplexConv2d(256, 512, kernel_size=3, padding=1),
          LeakyReLUComplex(negative_slope=0.1),
          ComplexConv2d(512, 256, kernel_size=1),
          ComplexConv2d(256, 512, kernel_size=3, padding=1),
          LeakyReLUComplex(negative_slope=0.1),
          ComplexConv2d(512, 512, kernel_size=3, padding=1),
          LeakyReLUComplex(negative_slope=0.1),
          ComplexConv2d(512, 512, kernel_size=3, stride=2, padding=1),
          LeakyReLUComplex(negative_slope=0.1),

          ComplexConv2d(512, 512, kernel_size=3, padding=1),
          LeakyReLUComplex(negative_slope=0.1),
          ComplexConv2d(512, 512, kernel_size=3, padding=1),
          LeakyReLUComplex(negative_slope=0.1),

          nn.Flatten()]

      layer6 = [ComplexLinear(7 * 7 * 512, 2048),
          ComplexDropout(),

          LeakyReLUComplex(negative_slope=0.1),

          ComplexLinear(2048, 7 * 7 * depth)
      ]

      self.layer1 = nn.Sequential(*layer1)
      self.layer2 = nn.Sequential(*layer2)
      self.layer3 = nn.Sequential(*layer3)
      self.layer4 = nn.Sequential(*layer4)
      self.layer5 = nn.Sequential(*layer5)
      self.layer6 = nn.Sequential(*layer6)

    def forward(self, X):
      x = self.layer1(X)
      # print(1, x)
      x = complex_max_pool2d(x, kernel_size=2, stride=2)
      # x = self.clampLayer(x)
      # print(2, x)
      x = self.layer2(x)
      # x = self.clampLayer(x)
      # print(3, x)
      x = complex_max_pool2d(x, kernel_size=2, stride=2)
      # x = self.clampLayer(x)
      # print(4, x)
      x = self.layer3(x)
      # x = self.clampLayer(x)
      # print(5, x)
      x = complex_max_pool2d(x, kernel_size=2, stride=2)
      # x = self.clampLayer(x)
      # print(6, x)
      x = self.layer4(x)
      # x = self.clampLayer(x)
      # print(7, x)
      x = complex_max_pool2d(x, kernel_size=2, stride=2)
      # x = self.clampLayer(x)
      # print(8, x.shape)
      x = self.layer5(x)
      # print(9, x.shape)
      x = self.layer6(x)
      # x = self.clampLayer(x)
      # print(9, x)
      x = x.abs()

      # print(10, x)
      return torch.reshape(x ,(X.shape[0], 7, 7, depth))

In [ ]:
class YOLOv1Loss(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, Y_pred, Y):
        batch_size = Y.size(0)
        loss = 0

        for i in range(batch_size):
            for y in range(grid_size):
                for x in range(grid_size):

                  loss += 0.5 * Y[i, x, y, 0] * ((Y_pred[i, x, y, 1] - Y[i, x, y, 1])**2 + (Y_pred[i, x, y, 2] - Y[i, x, y, 2])**2)
                  loss += 0.5 * Y[i, x, y, 5] * ((Y_pred[i, x, y, 6] - Y[i, x, y, 6])**2 + (Y_pred[i, x, y, 7] - Y[i, x, y, 7])**2)

                  loss += 0.5 * Y[i, x, y, 0] * ((abs(Y_pred[i, x, y, 3])**0.5 - Y[i, x, y, 3]**0.5)**2 + (abs(Y_pred[i, x, y, 4])**0.5 - Y[i, x, y, 4]**0.5)**2)
                  loss += 0.5 * Y[i, x, y, 5] * ((abs(Y_pred[i, x, y, 8])**0.5 - Y[i, x, y, 8]**0.5)**2 + (abs(Y_pred[i, x, y, 9])**0.5 - Y[i, x, y, 9]**0.5)**2)

                  if(Y[i, x, y, 0]==0):
                    loss += 0.5 * (Y[i, x, y, 0] - Y_pred[i, x, y, 0])**2
                  else:
                    loss += (Y[i, x, y, 0] - Y_pred[i, x, y, 0])**2

                  if(Y[i, x, y, 5]==0):
                    loss += 0.5 * (Y[i, x, y, 5] - Y_pred[i, x, y, 5])**2
                  else:
                    loss += (Y[i, x, y, 5] - Y_pred[i, x, y, 5])**2

                  for j in range(10, 10+num_classes):
                    loss += Y[i, x, y, 0] * ((Y[i, x, y, j]-Y_pred[i, x, y, j])**2)
        return loss

In [ ]:
model = YOLOv1().to(device)

In [ ]:
loss_function = YOLOv1Loss()

In [ ]:
optimizer = torch.optim.SGD(
        model.parameters(),
        lr=1e-4,
        momentum=0.9,
        weight_decay=0.0005
    )

In [ ]:
validation_loss = torch.load('/content/drive/MyDrive/VOC2007/Validation/complex_loss.pt').to(device)
validation_loss = list(validation_loss)

In [ ]:
training_loss = torch.load('/content/drive/MyDrive/VOC2007/Train/complex_loss.pt').to(device)
training_loss = list(training_loss)

In [ ]:
train_loader = DataLoader(TensorDataset(torch.load("/content/drive/MyDrive/VOC2007/Train/Batch3/train_ihsv.pt").to(device), torch.load("/content/drive/MyDrive/VOC2007/Train/Batch3/train_gt.pt").to(device)), batch_size=Batch_size)

In [ ]:
val_loader = DataLoader(TensorDataset(torch.load("/content/drive/MyDrive/VOC2007/Validation/Batch1/val_ihsv.pt").to(device), torch.load("/content/drive/MyDrive/VOC2007/Validation/Batch1/val_gt.pt").to(device)), batch_size=Batch_size)

In [ ]:
model.load_state_dict(torch.load('/content/drive/MyDrive/VOC2007/complex_model_weights.pt'))

## Epochs: 10

In [ ]:
for epoch in range(1, num_epoches):

  model.train()
  train_loss = 0
  for batch_idx, (X, Y) in enumerate(train_loader):

    optimizer.zero_grad()
    X = torch.permute(X, (0, 3, 1, 2))
    predictions = model.forward(X)
    predictions = torch.clamp(predictions, min=0, max=1)
    Loss = loss_function(predictions, Y)
    Loss.backward()
    optimizer.step()
    train_loss += Loss

  torch.save(model.state_dict(), '/content/drive/MyDrive/VOC2007/complex_model_weights.pt')
  print("training loss: epoch: ", epoch, "loss: ", train_loss/1500)
  training_loss.append(train_loss/1500)
  torch.save(torch.Tensor(training_loss).to(device), '/content/drive/MyDrive/VOC2007/Train/complex_loss.pt')

  model.eval()
  val_loss = 0
  with torch.inference_mode():
    for batch_idx, (X, Y) in enumerate(val_loader):
      X = torch.permute(X, (0, 3, 1, 2))
      predictions = model(X)
      val_loss += loss_function(predictions, Y)

  print("validation loss: epoch: ", epoch, "loss: ", val_loss/100)
  validation_loss.append(val_loss/100)
  torch.save(torch.Tensor(validation_loss).to(device), '/content/drive/MyDrive/VOC2007/Validation/complex_loss.pt')